# Walkthrough 1: Running Local Large Language Models (LLMs)

A key risks raised repeatedly within the challenge was that of Data Privacy when using an LLM. When we interact with an LLM we sometimes need to provide additional context which may include data that your company does not want to be sent over the internet to a 3rd party. While companies such as OpenAI (ChatGPT) and Microsoft (Bing Copilot) allow you to opt out from your data being used in training many companies prefer to remove the risk by using Open-Source LLMs locally.

In recent years there has been a growth in Open Source LLMs (such as Llama 2 from Meta or BERT from Google) - they use similar architectures to ChatGPT and Bing Copilot and share similar approaches to training the models.  These allow companies, if they want, to run LLMs locally.

Running an LLM locally does have some drawbacks:
* Generally larger LLM models will perform better but will require greater compute (memory and CPU/GPU) to be performant.
* The company needs to provide the infrastructure to host and run the model
* As the models evolve, the companies need to manage the model upgrades

However, running local LLMS models have a number of key benefits:
* Enhanced Data Security and Privacy since no data is sent to 3rd parties
* Cost saving and reduction in vendor lock in
* Ability to customise the LLM for their purposes

This walkthrough will show you 2 ways to do this:
* Using LlamaFile
* Using HuggingFace

> NOTE: This is not an exhaustive guide to deploying LLMs locally, instead it is to show you what is possible 


# Related Resources:

| Link                                               | Description                                                                                                                                             |
|----------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| https://www.datacamp.com/blog/top-open-source-llms | Introductory article on using Open Source LLMs                                                                                                          |
| https://huggingface.co/models| Link to the HuggingFace Pre-trained Models Hub|
| https://huggingface.co/learn/nlp-course            | A free course provided by HuggingFace that gets you up to speed with Natural Language Processing using the HuggingFace library (requires Python knowledge |


## Using LlamaFile
If you like the conversational style of tools such as Bing Copilot or ChatGPT you can host your own LLM in your desktop or a local server using an interesting project from Mozilla (https://www.mozilla.org/) called **LlamaFile**.

The **LlmaFile** project aims to package up Open-Source Large Language Models into an executable that can be run as local webserver with a simple Chat Inferface and an API for you to query.

The project can be found at https://github.com/Mozilla-Ocho/llamafile

The project's ReadMe file contains instructions on how you can download an image and get it to run on your local machine.

There are a few points to remember:
* The LLM models you can access are limited to Open-Source models - you won't find models such as GTP3.5, GPT4 available in LlamaFile as these are closed source.
* Large Language Models take a large amount of memory, so it's unlikely that you will be able to run a model the size of GPT on your desktop and so performance may not be as good. 
* The text generation may be slow depending on the power of your local machine.

> IMPORTANT: You will need to be able to run arbitrary executables on your machine. 
> Some company IT Security may prohibit this so please check before downloading and attempting to run the LlamaFile on company machines. 

For today's task I would suggest:
1. Pick one of the smaller models liked on LlamaFile.
2. Download the file and follow the instructions to run the LlamaFile
3. Explore some conversations with your LLM.


## Using LLMs programmatically using HuggingFace Models

During the challenge we used LLMs by typing in our prompts and we enhanced our prompts through various Prompt Engineering methods. Writing prompts into a chat style interface is certainly one way of interacting with a LLM, however it is not the only way we could use LLMs.

Imagine you've crafted a prompt to assess the quality of a defect report, perhaps it scores the defect report in terms of clarity and completeness and if the defect is lacking the LLM outputs a set of questions for the defect author. 

This might be a useful addition to your defect handling workflow but let's face it, if you needed to type (or copy) the prompt and the defect description into a Chat Interface for each defect it will quickly become tedious.

Instead, we can programmatically extract any new defects and for each one call an LLM to evaluate the defect report. We can achieve this in a straightforward manner using LLM from HuggingFace and a bit of Python code.

> For this walkthrough, you do not need to write any code. The code presented here is complete and designed to show we could integrate AI models. If you want to learn more about the coding aspect of integrating AI models then I would suggest you start with the HuggingFace NLP Course.

HuggingFace (https://huggingface.co) is a machine learning community that collaborates on building models and developing datasets. 

First we need to install some dependencies

In [ ]:
!pip install transformers bitsandbytes>=0.39.0 accelerate -q

Next we will use the Huggingface Transformer Library to create a pre-trained model based on the Open-Source Mistral-7B model.

When using HuggingFace, this is all the code that is needed!
This will download the model (this may take some time to complete) 

In [9]:
from transformers import AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-v0.1" 

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

If you remember when you created your prompts earlier in the challenge, you used fairly natural langauge to specify what you wanted.

However, machines don't really understand text, they only deal with numbers so we need to perform a task called Tokenization, which converts the text we type into a numerical form that the model can process. Moreover, the method we use to tokenize our text needs to be one that the model understands.

Tokenization is an involved process but again, HuggingFace make this really easy for us and with just a few lines of code we can tokenize sentences.   

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

To understand what the Tokenizer does, you can run the following cell with some text to see what the model receives as input

In [4]:
sample_input = "I've nearly completed the Ministry of Testing 30 Days of AI in Testing challenge"
sample_model_input = model_inputs = tokenizer([sample_input], return_tensors="pt")

print(sample_model_input)


{'input_ids': tensor([[    1,   315, 28742,   333,  5597,  7368,   272, 17036,   302,  3735,
           288, 28705, 28770, 28734, 19503,   302, 16107,   297,  3735,   288,
          8035]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


As you can see from the above output, our text gets converted into a large set of numbers.

To interact with the LLM model we need to provide our prompt and call the *generate()* method for the model.
The model will output a *tokenized* version of the output that we then need to decode back into text. 

Yet again, HuggingFace makes this a straightforward task. For convince we've wrapped all the HuggingFace code for generation into the following function.

In [5]:
def get_model_response(model, tokenizer,  model_prompt) -> str:
    """"
    This function wraps the calls to the tokenizer to tokenize the model_prompt, calls the model's generate function then decodes the response.
    """
    tokens = tokenizer([sample_input], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**tokens)
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

Now we want to generate some output. You can change the prompt text to be whatever you want. 
You can also change the text and re-run the cell as many times as you want.

In [6]:
prompt_text = "What are the key risks associated with using AI for decision making?"

print(get_model_response(model, tokenizer, prompt_text))

NameError: name 'model' is not defined

And that is it - using libraries such as HuggingFace can simplify the building of AI empowered tools. 
The simple and versatile programming library along with a large (and ever growing) pre-trained model hub is a powerful combination.

If you want to learn more about building using HuggingFace, I would recommend the HuggingFace NLP Course (https://huggingface.co/learn/nlp-course)
